In [368]:
import pandas as pd
import numpy as np

In [369]:
# Read Excel file into a DataFrame.
df = pd.read_excel('data.xlsx')

# Remove column.
df = df.drop(columns=['Unnamed: 0'])

# Change column name.
df = df.rename(columns={'Unnamed: 19': 'Reference'})

# Create column of reference with A and I.
df_ref_column = df['Reference']

# Store the DataFrame reference of A's and I's in a int.
total_A = df["Reference"].value_counts()["A"]
total_I = df["Reference"].value_counts()["I"]

# Define operative DataFrame:
# with the columns from X11 to X92, without Reference column.
df_op = df.loc[:, "X11": "X92"]

# Count the number of operatives columns (between X11 and X92).
nbr_operatives_columns = len(df_op.columns)

df.head()

,X11,X12,X21,X22,X31,X32,X41,X42,X51,X52,X61,X62,X71,X72,X81,X82,X91,X92,Reference
0,1,0,0,1,1,0,0,1,0,1,1,0,0,1,1,0,1,0,A
1,0,1,0,1,1,0,0,1,0,1,1,0,0,1,1,0,1,0,A
2,1,0,1,0,0,1,1,0,1,0,0,1,1,0,0,1,0,1,I
3,1,0,0,1,0,1,1,0,1,0,0,1,1,0,0,1,0,1,I
4,0,1,0,1,1,0,0,1,0,1,1,0,0,1,0,1,1,0,A


In [370]:
# Create firstborns (chromosomes).

# Set the seed for reproducibility
#np.random.seed(42)

# Generate 6 random chromosomes.
chromosome1 = -1 + 2 * np.random.random(nbr_operatives_columns)
chromosome2 = -1 + 2 * np.random.random(nbr_operatives_columns)
chromosome3 = -1 + 2 * np.random.random(nbr_operatives_columns)
chromosome4 = -1 + 2 * np.random.random(nbr_operatives_columns)
chromosome5 = -1 + 2 * np.random.random(nbr_operatives_columns)
chromosome6 = -1 + 2 * np.random.random(nbr_operatives_columns)

# Create re-usable DataFrame with the current lineage
df_current_lineage = pd.DataFrame({'chromo_1':chromosome1,
								   'chromo_2':chromosome2,
								   'chromo_3':chromosome3,
								   'chromo_4':chromosome4,
								   'chromo_5':chromosome5,
								   'chromo_6':chromosome6})

df_current_lineage.head()

,chromo_1,chromo_2,chromo_3,chromo_4,chromo_5,chromo_6
0,-0.067717,-0.612185,-0.209472,0.258140,0.132886,0.680376
1,0.210418,0.318868,-0.440653,-0.334808,-0.979154,0.474883
2,-0.334890,-0.604227,0.362926,-0.027371,-0.443958,0.624744
3,-0.892480,0.137364,0.294272,-0.516307,0.419700,-0.286113
4,0.185799,0.117919,0.876265,-0.848489,0.418612,-0.816645


In [371]:
# Function to aply each chromosome for each line of the DataFrame
# and return the result (line * chromosome).sum().
def chromo_action(row, chromosome, trash):
	res_mult = row * chromosome
	res_sum = res_mult.sum()
	return (res_sum)

# Iterate df_current_lineage in database.
i = 0
trash = None
df_chromo_result = pd.DataFrame()
max_i = len(df_current_lineage.columns)
while i < max_i:
	current_chromosome = (df_current_lineage.iloc[:, i]).values
	chromo_result = df_op.apply(func=chromo_action, axis=1, args=(current_chromosome, trash))
	df_chromo_result[f'Chromo {i+1} result'] = chromo_result
	i = i + 1

# Create chromo reference: a DataFrame that contain
# 'A' if number > 0 and 'I' if number < 0.
df_chromo_result = df_chromo_result.applymap(lambda x: 'A' if x > 0 else 'I')

# Calculate how many A's and how many I's were correct.
def chromo_count_matches(column):
	relative_A = ((column == 'A') & (df_ref_column == 'A')).sum()
	relative_I = ((column == 'I') & (df_ref_column == 'I')).sum()
	return pd.Series({'Right A': relative_A, 'Right I': relative_I})

df_chromo_result = df_chromo_result.apply(chromo_count_matches)

# Calculate the fitness for all the six chromosome
def fitness_calculation(column):
	relative_numerator = np.prod(column)
	fitness = relative_numerator / (total_A * total_I)
	return fitness

fitness_values = df_chromo_result.apply(fitness_calculation)
df_chromo_result.loc['Fitness'] = fitness_values

# Removing obsolete A and I rows
i_remove = ['Right A', 'Right I']
df_chromo_result = df_chromo_result.drop(i_remove)

df_chromo_result

,Chromo 1 result,Chromo 2 result,Chromo 3 result,Chromo 4 result,Chromo 5 result,Chromo 6 result
Fitness,0.120614,0.541667,0.131579,0.013158,0.0,0.017544


## Raffle

In [372]:
# Get two raffle points to choose two chromosomes.
# The choosen chromosomes are going to be crossed.

# Create array with fitness values.
fitness_array = (df_chromo_result.iloc[0, :]).values

# Sum the array.
fitness_array_sum = fitness_array.sum()

# Create new array with cumulative sum.
norm_fit_0 = round((fitness_array[0]/ fitness_array_sum) * 100)
norm_fit_1 = round((fitness_array[1]/ fitness_array_sum) * 100) + norm_fit_0
norm_fit_2 = round((fitness_array[2]/ fitness_array_sum) * 100) + norm_fit_1
norm_fit_3 = round((fitness_array[3]/ fitness_array_sum) * 100) + norm_fit_2
norm_fit_4 = round((fitness_array[4]/ fitness_array_sum) * 100) + norm_fit_3
norm_fit_5 = round((fitness_array[5]/ fitness_array_sum) * 100) + norm_fit_4
norm_fitness_array = np.array([norm_fit_0, norm_fit_1, norm_fit_2, norm_fit_3, norm_fit_4, norm_fit_5])

# Sum the new array (this value must be equal to one).
# This is not used, is just a check.
norm_fitness_array_sum = norm_fitness_array.sum()

# raffle function: it resturn a position given a raffle number.
def get_raffle_point(raffle):
	max_i = len(norm_fitness_array)
	i = 0
	while i < max_i:
		if 0 <= raffle <= norm_fitness_array[0]:
			return 0
		elif norm_fitness_array[i] < raffle <= norm_fitness_array[i + 1]:
			return i
		i = i + 1

# Get two raffle points given a random raffle.
raffle_1 = round(np.random.random() * 100)
raffle_point_1 = get_raffle_point(raffle_1)
raffle_2 = round(np.random.random() * 100)
raffle_point_2 = get_raffle_point(raffle_2)

# Selec (from the random above) a mother and a father chromosome.
father_chromosome = df_current_lineage.iloc[:, raffle_point_1]
mother_chromosome = df_current_lineage.iloc[:, raffle_point_2]

## Cross

In [373]:
father_chromosome.values

array([-0.06771656,  0.21041832, -0.33489015, -0.89248023,  0.18579903,
       -0.2118521 , -0.28737043, -0.45022243, -0.03470813, -0.38118932,
        0.35688531,  0.19865922,  0.00959741, -0.38759736,  0.88028436,
        0.25519212,  0.33136279,  0.9429483 ])

In [374]:
mother_chromosome.values

array([-0.06771656,  0.21041832, -0.33489015, -0.89248023,  0.18579903,
       -0.2118521 , -0.28737043, -0.45022243, -0.03470813, -0.38118932,
        0.35688531,  0.19865922,  0.00959741, -0.38759736,  0.88028436,
        0.25519212,  0.33136279,  0.9429483 ])

In [375]:
def cross_father_mother(father, mother):
	cross_point = round(17 * np.random.random())
	paternal_sperm  = father[cross_point:]
	maternal_egg = mother[:cross_point]
	new_born = np.hstack((maternal_egg, paternal_sperm))
	return (new_born)

new_born_1 = cross_father_mother(father_chromosome.values, mother_chromosome.values)
new_born_2 = cross_father_mother(father_chromosome.values, mother_chromosome.values)
new_born_3 = cross_father_mother(father_chromosome.values, mother_chromosome.values)

print(new_born_1)
print()
print(new_born_2)
print()
print(new_born_3)

[-0.06771656  0.21041832 -0.33489015 -0.89248023  0.18579903 -0.2118521
 -0.28737043 -0.45022243 -0.03470813 -0.38118932  0.35688531  0.19865922
  0.00959741 -0.38759736  0.88028436  0.25519212  0.33136279  0.9429483 ]

[-0.06771656  0.21041832 -0.33489015 -0.89248023  0.18579903 -0.2118521
 -0.28737043 -0.45022243 -0.03470813 -0.38118932  0.35688531  0.19865922
  0.00959741 -0.38759736  0.88028436  0.25519212  0.33136279  0.9429483 ]

[-0.06771656  0.21041832 -0.33489015 -0.89248023  0.18579903 -0.2118521
 -0.28737043 -0.45022243 -0.03470813 -0.38118932  0.35688531  0.19865922
  0.00959741 -0.38759736  0.88028436  0.25519212  0.33136279  0.9429483 ]
